In [1]:
import torchvision
import torchvision.transforms as transforms
import os
import yaml
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

os.chdir('../')
from utils import Config
from models.ViT import vit_tiny

/home/coartix/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_hip.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
with open("configs/cifar10_base.yml") as f:
        yml_dict = yaml.load(f, Loader=yaml.FullLoader)
config = Config(yml_dict)

In [3]:
dataset_original = torchvision.datasets.CIFAR10(
    root=config.original_dataset_path,
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

Files already downloaded and verified


### Testing ViT on classification task

In [4]:
from models.ViT import vit_tiny
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [5]:
clf_model = vit_tiny(patch_size=4, embed_dim=192, depth=6, num_classes=10, img_size=[32],
  drop_rate=0.2, attn_drop_rate=0.2, drop_layer_rate=0.2, num_heads=6)
clf_model.cuda()

ViT_small(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
  )
  (pos_drop): Dropout(p=0.2, inplace=False)
  (blocks): ModuleList(
    (0-4): 5 x Block(
      (norm1): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
      (attn): SelfAttention(
        (q): Linear(in_features=192, out_features=192, bias=True)
        (k): Linear(in_features=192, out_features=192, bias=True)
        (v): Linear(in_features=192, out_features=192, bias=True)
        (attn_drop): Dropout(p=0.2, inplace=False)
        (proj): Linear(in_features=192, out_features=192, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (norm2): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
      (mlp): MLP(
        (fc1): Linear(in_features=192, out_features=768, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=768, out_features=192, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (drop_

Creating dataloader

In [6]:
train_size = int(0.8 * len(dataset_original))
test_size = len(dataset_original) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset_original, [train_size, test_size])

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=config.batch_size,
    shuffle=True
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=config.batch_size,
    shuffle=False
)

Simple training

In [7]:
epochs = 20
optimizer = optim.Adam(clf_model.parameters())
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
    print("Epoch:", epoch)
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        inputs, label = data
        inputs, label = inputs.cuda(), label.cuda()
        output = clf_model(inputs)

        optimizer.zero_grad()
        loss = F.cross_entropy(output, label)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        if i % 25 == 24:
            print("Loss:", running_loss / 25)
            running_loss = 0.0

print("Finished Training")

Epoch: 0
Loss: 2.347198123931885
Loss: 2.2167662239074706
Epoch: 1
Loss: 2.040811152458191
Loss: 2.002656092643738
Epoch: 2
Loss: 1.9329124116897582
Loss: 1.8971667718887328
Epoch: 3
Loss: 1.880981822013855
Loss: 1.8420228624343873
Epoch: 4
Loss: 1.8387811803817748
Loss: 1.8184457969665528
Epoch: 5
Loss: 1.763266043663025
Loss: 1.7365613269805908
Epoch: 6
Loss: 1.750727548599243
Loss: 1.7459416007995605
Epoch: 7
Loss: 1.6914555358886718
Loss: 1.7028889989852904
Epoch: 8
Loss: 1.6671459197998046
Loss: 1.6606989049911498
Epoch: 9
Loss: 1.6259217596054076
Loss: 1.6596532583236694
Epoch: 10
Loss: 1.6262207746505737
Loss: 1.6141299772262574
Epoch: 11
Loss: 1.6079702854156495
Loss: 1.5789149522781372
Epoch: 12
Loss: 1.604694504737854
Loss: 1.609003086090088
Epoch: 13
Loss: 1.5666968774795533
Loss: 1.5651464653015137
Epoch: 14
Loss: 1.5299469089508058
Loss: 1.5822199201583862
Epoch: 15
Loss: 1.5323998880386354
Loss: 1.5439596462249756
Epoch: 16
Loss: 1.5201569032669067
Loss: 1.544232888221740

Testing

In [9]:
correct = 0
total = 0

with torch.no_grad():
    for data in test_dataloader:
        input, label = data
        input, label = input.cuda(), label.cuda()
        output = clf_model(input)
        _, predicted = torch.max(output.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()

print(f"Accuracy of the network on the test images: {100 * correct / total}%")

Accuracy of the network on the test images: 43.7%
